> # **Great**

In [ ]:
import json
import math
import os
import numpy
import cv2
from PIL import Image
import numpy as np
from keras import layers
from keras.applications import DenseNet121
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
import scipy
import tensorflow as tf
from tqdm import tqdm

%matplotlib inline

Set random seed for reproducibility.

In [ ]:
np.random.seed(420)
tf.set_random_seed(420)

Load the data and take a look at the format

In [ ]:
train_df = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')
print(train_df.shape)
train_df.head()

In [ ]:
train_df['diagnosis'].hist()

Let's balance the samples to remove any bias in the end result.

Because diagnosis: 3 has the least samples, we'll create a new dataframe with a random sampling for each diagnosis category with the same number of samples in category 3.

In [ ]:
new_train_df = train_df[train_df['diagnosis'] == 3]
cat_size = len(new_train_df)
new_train_df = new_train_df.append(train_df[train_df['diagnosis'] == 0].sample(n = cat_size), ignore_index=True)
new_train_df = new_train_df.append(train_df[train_df['diagnosis'] == 1].sample(n = cat_size), ignore_index=True)
new_train_df = new_train_df.append(train_df[train_df['diagnosis'] == 2].sample(n = cat_size), ignore_index=True)
new_train_df = new_train_df.append(train_df[train_df['diagnosis'] == 4].sample(n = cat_size), ignore_index=True)

#construct a dataframe containing all the rows NOT included in the train_df...
test_df = train_df.copy()
test_df = test_df.loc[~test_df['id_code'].isin(new_train_df['id_code'])].reset_index(drop=True)

train_df = new_train_df

train_df['diagnosis'].hist()

train_df = train_df.sample(frac = 1).reset_index(drop=True)

### Displaying some Sample Images

In [ ]:
def display_samples(df, columns=4, rows=3):
    fig=plt.figure(figsize=(5*columns, 4*rows))

    for i in range(columns*rows):
        image_path = df.loc[i,'id_code']
        image_id = df.loc[i,'diagnosis']
        img = cv2.imread(f'../input/aptos2019-blindness-detection/train_images/{image_path}.png')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        fig.add_subplot(rows, columns, i+1)
        plt.title(image_id)
        plt.imshow(img)
        plt.axis('off')
    
    plt.tight_layout()

display_samples(train_df)

# Resize Images

We will resize the images to 224x224, then create a single numpy array to hold the data.

In [ ]:
def preprocess_image(image_path, desired_size=224):
    sigmaX = 10
    im = Image.open(image_path)
    im = im.resize((desired_size, )*2, resample=Image.LANCZOS)
    opencv_image = numpy.array(im) 
    opencv_image = cv2.addWeighted(opencv_image, 4, cv2.GaussianBlur(opencv_image, (0, 0), sigmaX), -4, 128)
    
    return opencv_image[:, :, ::-1].copy() 

In [ ]:
def display_samples(df, columns=4, rows=3):
    fig=plt.figure(figsize=(5*columns, 4*rows))

    for i in range(columns*rows):
        image_path = df.loc[i,'id_code']
        image_id = df.loc[i,'diagnosis']
        fig.add_subplot(rows, columns, i+1)
        plt.title(image_id)
        plt.imshow(preprocess_image(f'../input/aptos2019-blindness-detection/train_images/{image_path}.png'))
        plt.axis('off')
    
    plt.tight_layout()

display_samples(train_df)

In [ ]:
N = train_df.shape[0]
x_train = np.empty((N, 224, 224, 3), dtype=np.uint8)

for i, image_id in enumerate(tqdm(train_df['id_code'])):
    x_train[i, :, :, :] = preprocess_image(
        f'../input/aptos2019-blindness-detection/train_images/{image_id}.png'
    )

In [ ]:
N = test_df.shape[0]
x_test = np.empty((N, 224, 224, 3), dtype=np.uint8)

for i, image_id in enumerate(tqdm(test_df['id_code'])):
    x_test[i, :, :, :] = preprocess_image(
        f'../input/aptos2019-blindness-detection/train_images/{image_id}.png'
    )

In [ ]:
y_train = pd.get_dummies(train_df['diagnosis']).values

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)

## Creating multilabels

Instead of predicting a single label, we will change our target to be a multilabel problem; i.e., if the target is a certain class, then it encompasses all the classes before it. E.g. encoding a class 4 retinopathy would usually be `[0, 0, 0, 1]`, but in our case we will predict `[1, 1, 1, 1]`. For more details, please check out [Lex's kernel](https://www.kaggle.com/lextoumbourou/blindness-detection-resnet34-ordinal-targets).

In [ ]:
y_train_multi = np.empty(y_train.shape, dtype=y_train.dtype)
y_train_multi[:, 4] = y_train[:, 4]
for i in range(3, -1, -1):
    y_train_multi[:, i] = np.logical_or(y_train[:, i], y_train_multi[:, i+1])

print("Original y_train:", y_train.sum(axis=0))
print("Multilabel version:", y_train_multi.sum(axis=0))

Now we can split it into a training and validation set.

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train_multi, 
    test_size=0.15, 
    random_state=420
)

In [ ]:
BATCH_SIZE = 32

def create_datagen():
    return ImageDataGenerator(
        zoom_range=0.15,  # set range for random zoom
        # set mode for filling points outside the input boundaries
        fill_mode='constant',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True,  # randomly flip images
    )

data_generator = create_datagen().flow(x_train, y_train, batch_size=BATCH_SIZE, seed=420)

### Creating keras callback for QWK

In [ ]:
class Metrics(Callback):
    def on_train_begin(self, logs={}):
        self.val_kappas = []

    def on_epoch_end(self, epoch, logs={}):
        X_val, y_val = self.validation_data[:2]
        y_val = y_val.sum(axis=1) - 1
        
        y_pred = self.model.predict(X_val) > 0.5
        y_pred = y_pred.astype(int).sum(axis=1) - 1

        _val_kappa = cohen_kappa_score(
            y_val,
            y_pred, 
            weights='quadratic'
        )

        self.val_kappas.append(_val_kappa)

        print(f"val_kappa: {_val_kappa:.4f}")
        
        if _val_kappa == max(self.val_kappas):
            print("Validation Kappa has improved. Saving model.")
            self.model.save('model.h5')

        return

# Model: DenseNet-121

In [ ]:
densenet = DenseNet121(
    weights='../input/densenet-keras/DenseNet-BC-121-32-no-top.h5',
    include_top=False,
    input_shape=(224,224,3)
)

In [ ]:
def build_model():
    model = Sequential()
    model.add(densenet)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(5, activation='sigmoid'))
    
    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(lr=0.00005),
        metrics=['accuracy']
    )
    
    return model

In [ ]:
model = build_model()
model.summary()

# Training & Evaluation

In [ ]:
kappa_metrics = Metrics()

history = model.fit_generator(
    data_generator,
    steps_per_epoch=x_train.shape[0] / BATCH_SIZE,
    epochs=15,
    validation_data=(x_val, y_val),
    callbacks=[kappa_metrics]
)

In [ ]:
with open('history.json', 'w') as f:
    json.dump(history.history, f)

history_df = pd.DataFrame(history.history)
history_df[['loss', 'val_loss']].plot()
history_df[['acc', 'val_acc']].plot()

In [ ]:
plt.plot(kappa_metrics.val_kappas)

## Find best threshold

Please Note: Although I show how to construct a threshold optimizer, **it is currently unused**. Please see notice at the top of the kernel.

In [ ]:
model.load_weights('model.h5')
y_val_pred = model.predict(x_val)

def compute_score_inv(threshold):
    y1 = y_val_pred > threshold
    y1 = y1.astype(int).sum(axis=1) - 1
    y2 = y_val.sum(axis=1) - 1
    score = cohen_kappa_score(y1, y2, weights='quadratic')
    
    return 1 - score

simplex = scipy.optimize.minimize(
    compute_score_inv, 0.5, method='nelder-mead'
)

best_threshold = simplex['x'][0]

## Submit

In [ ]:
y_test = model.predict(x_test) > 0.5
y_test = y_test.astype(int).sum(axis=1) - 1
output_df = test_df.copy()
output_df['prediction'] = y_test
print(accuracy_score(output_df['diagnosis'].tolist(), output_df['prediction'].tolist()))
output_df.to_csv('output.csv',index=False)